# 정제된 주소 만들기
PNU 만들기
1. 주소 칼럼 정리
2. 주소가 유효한지 확인
3. 주소별 숫자매칭하여 PNU 작성

[1] 주소 → API 요청 → 응답 결과 확인  
       └─ 결과가 있음 → 유효 주소로 판단  
       └─ 결과 없음 → 오류/오타 주소  

[2] 응답에서 admCd(법정동코드), 본번, 부번, 산 여부 추출

[3] PNU 생성:
       PNU = 법정동코드(10) + 산여부(1) + 본번(4) + 부번(4)


## 주소 칼럼 정리

In [2]:
import pandas as pd
import re

# 1. 엑셀에서 원본 주소 불러오기
df = pd.read_excel("0_주소원본.xlsx")
rows = df.iloc[:, 0].dropna().astype(str).tolist()  # 첫 열만 추출 + 문자열 처리

result = []

# 2. 각 행 반복 처리
for row in rows:
    row = row.strip()
    
    if ',' in row:
        # 쉼표 기준으로 나눔
        parts = [part.strip() for part in row.split(',')]
        
        # 첫 번째 항목에서 prefix 추출
        first_part = parts[0]
        match = re.search(r'\d', first_part)
        if match:
            prefix = first_part[:match.start()].strip()
            numbers = [first_part[match.start():]] + parts[1:]
            for num in numbers:
                result.append(f"{prefix} {num}")
        else:
            result.append(row)  # 숫자 없으면 원래 값 유지
    else:
        result.append(row)  # 쉼표 없는 경우 그대로

# 3. 결과 저장
df_result = pd.DataFrame(result, columns=["주소"])
df_result.to_excel("1_주소정리.xlsx", index=False)


## 주소 유효성 확인

In [4]:
import pandas as pd
import requests
import time
import re

# 🔑 API 설정
API_KEY = "972C991F-F755-3021-B74F-175B82AA82F8"  #vworld api 인증키 넣으시면 됩니다.
API_URL = "https://api.vworld.kr/req/search"

# 🧹 주소 전처리
def clean_address(addr):
    addr = str(addr).strip()
    addr = re.sub(r',$', '', addr)
    addr = re.sub(r'\s+', ' ', addr)
    return addr

# 📦 주소 존재 확인 (지번 기준)
def check_parcel_exists(addr):
    params = {
        "service": "search",
        "request": "search",
        "version": "2.0",
        "format": "json",
        "type": "address",
        "category": "parcel",
        "size": 1,
        "crs": "EPSG:4326",
        "key": API_KEY,
        "query": addr
    }
    try:
        res = requests.get(API_URL, params=params, timeout=5)
        data = res.json()
        items = data.get("response", {}).get("result", {}).get("items", [])
        exists = bool(items)

        return {
            "입력주소": addr,
            "존재여부": "✅ 있음" if exists else "❌ 없음"
        }
    except Exception as e:
        return {
            "입력주소": addr,
            "존재여부": f"❌ 오류({e})"
        }

# 📄 엑셀 불러오기
df = pd.read_excel("1_주소정리.xlsx")
cleaned_addresses = df["주소"].dropna().map(clean_address).tolist()

# 🔍 검사 반복
results = []
for addr in cleaned_addresses:
    result = check_parcel_exists(addr)
    results.append(result)
    time.sleep(0.1)

# 💾 전체 저장
df_result = pd.DataFrame(results)

# 💾 ❌ 없음 필터링 후 별도 저장
df_invalid = df_result[df_result["존재여부"] == "❌ 없음"]
df_invalid.to_excel("2_1_주소_유효성_미존재.xlsx", index=False)

# 💾 ✅ 있음(유효) 필터링 후 별도 저장
df_valid = df_result[df_result["존재여부"] == "✅ 있음"]
df_valid.to_excel("2_2_주소_유효성_존재.xlsx", index=False)


## PNU 작성

In [5]:
import pandas as pd
import re

# 1. 엑셀 파일 불러오기
df = pd.read_excel("2_2_주소_유효성_존재.xlsx")

# 2. 주소 분해 함수 정의
def split_jibun_address(addr):
    parts = addr.strip().split()
    # 1) 시도
    sido = next((p for p in parts if p.endswith(("도","특별시","광역시","특별자치시"))), "")
    
    # 2) 읍면동이 시작되는 인덱스 찾기
    idx_e = next(
        (i for i,p in enumerate(parts) if p.endswith(("읍","면","동"))),
        None
    )
    
    # 3) 시군구: 시도 바로 다음부터 읍면동 직전까지 
    sigungu_parts = []
    if idx_e is not None:
        for p in parts[1:idx_e]:
            if p.endswith(("시","군","구")):
                sigungu_parts.append(p)
    sigungu = " ".join(sigungu_parts)
    
    # 4) 읍면동: 읍면동 접미사
    eupmyeondong = parts[idx_e] if idx_e is not None else ""
    
    # 5) 리: 한자 괄호 포함 ‘리’ 토큰
    ri = next((p for p in parts if re.search(r'리(?:\(|$)', p)), "")
    
    # 6) 지번(본번/부번/산여부) 
    jibun = next((p for p in parts if re.match(r'^\d', p)), "")
    if jibun:
        산여부 = "산" if "산" in jibun else "일반"
        nums = re.sub(r"[^\d\-]", "", jibun).split("-",1)
        bon, bub = nums[0], nums[1] if len(nums)>1 else "0"
    else:
        산여부 = "일반"
        bon, bub = "0","0"
    본번 = re.sub(r"\D","",bon).zfill(4)
    부번 = re.sub(r"\D","",bub).zfill(4)
    
    return pd.Series({
        "시도": sido,
        "시군구": sigungu,
        "읍면동": eupmyeondong,
        "리": ri,
        "본번": 본번,
        "부번": 부번,
        "산여부": 산여부
    })

# 3. 적용
split_cols = df["입력주소"].apply(split_jibun_address)
df_result = pd.concat([df, split_cols], axis=1)

# 4. 저장
df_result.to_excel("3_주소_분해결과.xlsx", index=False)


In [6]:
import pandas as pd
import re

# 1. 주소 데이터 불러오기
df = pd.read_excel("3_주소_분해결과.xlsx")

# 2. 법정동코드 매핑 테이블 불러오기
code_df = pd.read_csv("0_법정동코드.csv", dtype=str)  # 주소, 코드 컬럼 필요

# 3. 필요한 컬럼만 추출해서 주소 키 생성
code_df['주소키'] = code_df[['시도명', '시군구명', '읍면동명','동리명']].fillna("").agg(" ".join, axis=1).str.strip()

# 4. 유효한 주소키만 필터링
valid_code_df = code_df[(code_df['주소키'] != "") & (code_df['법정동코드2'].notna())]

# 5. 기존 df에 주소키 만들기
df['주소키'] = df[['시도', '시군구', '읍면동','리']].fillna("").agg(" ".join, axis=1).str.strip()

# 6. PNU 생성 함수 수정 (법정동코드 테이블 기반)
def generate_pnu(row):
    try:
        주소키 = row['주소키']
        본번 = str(row['본번']).zfill(4)
        부번 = str(row['부번']).zfill(4)
        산여부 = row['산여부']

        match = valid_code_df[valid_code_df['주소키'] == 주소키]

        if match.empty:
            return "❌ 법정동코드 없음"

        법정동코드 = match.iloc[0]['법정동코드2']
        산코드 = "1" if 산여부 == "산" else "2"

        return f"{법정동코드}{산코드}{본번}{부번}"

    except Exception:
        return "❌ 오류 발생"

# 4. PNU 컬럼 생성
df["PNU"] = df.apply(generate_pnu, axis=1)

# 5. 저장
df.to_excel("4_PNU_생성결과.xlsx", index=False)